In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("oil_production.csv")
df.drop("Unnamed: 0", axis = 1, inplace = True)
df.drop("DATEPRD",axis = 1, inplace = True)

In [3]:
X = df.iloc[:,0:6]
#X = df[['AVG_DOWNHOLE_PRESSURE','AVG_DP_TUBING', 'AVG_CHOKE_SIZE_P',"AVG_WHP_P", "DP_CHOKE_SIZE" ]]
Y = df.iloc[:,8]

In [4]:
from sklearn.preprocessing import MinMaxScaler, Normalizer, MaxAbsScaler, StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, LassoCV, ElasticNet, SGDRegressor, Lasso

In [5]:
from sklearn.svm import SVR, LinearSVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor

In [6]:
import plotly.graph_objs as go
import plotly.offline as py
py.init_notebook_mode(connected= True)

In [7]:
test_fraction = 0.15
val_fraction = 0.15
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = test_fraction, shuffle = True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size 
                                                  = val_fraction/(1-test_fraction), shuffle = True)

In [8]:
scaler_x = MinMaxScaler()
x_train = scaler_x.fit_transform(x_train.as_matrix())
x_val = scaler_x.transform(x_val.as_matrix())
x_test = scaler_x.transform(x_test.as_matrix())

scaler_y = MinMaxScaler()
y_train = scaler_y.fit_transform(y_train.reshape(-1,1))
y_val = scaler_y.transform(y_val.reshape(-1,1))
y_test = scaler_y.transform(y_test.reshape(-1,1))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead



In [9]:
LR = LinearRegression()
LR.fit(x_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning:

internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.



LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [10]:
LR.score(x_val,y_val)

0.5139328448592981

In [11]:
LR.score(x_test,y_test)

0.5693971133857433

In [12]:
from sklearn.metrics import mean_squared_error as mse
from scipy.stats import pearsonr as r

In [13]:
print(mse(y_val, LR.predict(x_val)))
print(mse(y_test, LR.predict(x_test)))
r(y_test,LR.predict(x_test).reshape(-1,1))[0]

0.018111435809197017
0.019308258365908597


array([0.75820375])

In [14]:
y_act = scaler_y.inverse_transform(y_test).reshape(-1,1)

In [15]:
y_pred_LR = scaler_y.inverse_transform(LR.predict(x_test).reshape(-1,1))
trace1 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_LR.reshape(-1,),
                   name = "Actual",
                   mode = 'markers',
                   line=dict(width=10),
                   marker = dict(color = 'rgb(255,0,0)'))
data = [trace1]
layout = go.Layout(
    xaxis=dict(
        title='Actual water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='Predicted water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [16]:
MLP = MLPRegressor(hidden_layer_sizes=(50,),  activation='relu', solver='adam', alpha=0.0001, batch_size=200,
    learning_rate='adaptive', learning_rate_init=0.001, power_t=0.5, max_iter=1000, shuffle=True,
    random_state=9, tol=0.0001, verbose=False, warm_start=False, momentum=0.5, nesterovs_momentum=True,
    early_stopping=True, validation_fraction=0.15, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

In [17]:
MLP.fit(x_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



MLPRegressor(activation='relu', alpha=0.0001, batch_size=200, beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(50,), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=1000, momentum=0.5,
       nesterovs_momentum=True, power_t=0.5, random_state=9, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.15, verbose=False,
       warm_start=False)

In [18]:
print(MLP.score(x_val, y_val))
MLP.score(x_test, y_test)

0.6712478457493615


0.6816547623129982

In [19]:
from sklearn.metrics import mean_squared_error as mse
from scipy.stats import pearsonr as r

In [20]:
print(mse(y_val, MLP.predict(x_val)))
print(mse(y_test, MLP.predict(x_test)))
r(y_test, MLP.predict(x_test).reshape(-1,1))[0]

0.012249693228340275
0.014274618888757128


array([0.83371318])

In [21]:
y_pred_mlp = scaler_y.inverse_transform(MLP.predict(x_test).reshape(-1,1))
trace1 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_mlp.reshape(-1,),
                   name = "Actual",
                   mode = 'markers',
                   line=dict(width=10),
                   marker = dict(color = 'rgb(0,150,0)'))
data = [trace1]
layout = go.Layout(
    xaxis=dict(
        title='Actual water rate ',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='Predicted water rate',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [22]:
model_gbm = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [23]:
model_gbm.fit(x_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.05, loss='huber', max_depth=4,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=3000,
             presort='auto', random_state=5, subsample=1.0, verbose=0,
             warm_start=False)

In [24]:
print(model_gbm.score(x_val, y_val))
model_gbm.score(x_test, y_test)

0.9247443863926511


0.884932565071086

In [25]:
model_gbm.feature_importances_

array([0.15666487, 0.2070344 , 0.17121179, 0.13501096, 0.16288748,
       0.16719049])

In [26]:
from sklearn.metrics import mean_squared_error as mse
from scipy.stats import pearsonr as r
print(mse(y_val, model_gbm.predict(x_val)))
print(mse(y_test, model_gbm.predict(x_test)))
r(y_test, model_gbm.predict(x_test).reshape(-1,1))[0]

0.0028041129722840225
0.005159630444140846


array([0.94100923])

In [27]:
trace1 = go.Bar(y = model_gbm.feature_importances_,
               name = "Feaure importance")
data= [trace1]
layout = go.Layout(title = "Feature importances")
fig = go.Figure(data= data, layout = layout)
py.iplot(fig)

In [28]:
y_pred_gbm = scaler_y.inverse_transform(model_gbm.predict(x_test).reshape(-1,1))
trace1 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_gbm.reshape(-1,),
                   name = "Gradient Boosting Machine",
                   mode = 'markers',
                   line=dict(width=10),
                   marker = dict(
                    color = 'rgb(0, 150, 0)'))
data = [trace1]
layout = go.Layout(showlegend = True,
    xaxis=dict(
        title='Actual water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='Predicted water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [29]:
model_rf, model_ada = RandomForestRegressor(), AdaBoostRegressor()

In [30]:
model_rf.fit(x_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [31]:
y_pred_rf = scaler_y.inverse_transform(model_rf.predict(x_test).reshape(-1,1))
trace1 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_rf.reshape(-1,),
                   name = "Random Forest",
                   mode = 'markers',
                   line=dict(width=10),
                   marker = dict(
                    color = 'rgb(0, 150, 0)'))
data = [trace1]
layout = go.Layout(showlegend = True,
    xaxis=dict(
        title='Actual water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='Predicted water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [32]:
print(model_rf.score(x_val, y_val))
model_rf.score(x_test, y_test)

0.9168531655737735


0.849883201203238

In [33]:
from sklearn.metrics import mean_squared_error as mse
from scipy.stats import pearsonr as r
print(mse(y_val, model_rf.predict(x_val)))
print(mse(y_test, model_rf.predict(x_test)))
r(y_test, model_rf.predict(x_test).reshape(-1,1))[0]

0.0030981491724381555
0.006731245949188283


array([0.92230044])

In [34]:
model_ada.fit(x_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None)

In [35]:
y_pred_ada = scaler_y.inverse_transform(model_ada.predict(x_test).reshape(-1,1))
trace1 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_ada.reshape(-1,),
                   name = "Actual",
                   mode = 'markers',
                   line=dict(width=10),
                   marker = dict(
                    color = 'rgb(255, 0, 0)'))
data = [trace1]
layout = go.Layout(
    xaxis=dict(
        title='Actual water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='Predicted water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [36]:
print(model_ada.score(x_val, y_val))
print(model_ada.score(x_test, y_test))
print(mse(y_val, model_ada.predict(x_val)))
print(mse(y_test, model_ada.predict(x_test)))
r(y_test, model_rf.predict(x_test).reshape(-1,1))[0]

0.7180218682693811
0.762513573711672
0.010506837951142933
0.010648905104250003


array([0.92230044])

In [37]:
import xgboost as xgb
import lightgbm as lgb

model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [38]:
model_xgb.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4603, gamma=0.0468, learning_rate=0.05,
       max_delta_step=0, max_depth=3, min_child_weight=1.7817,
       missing=None, n_estimators=2200, n_jobs=1, nthread=-1,
       objective='reg:linear', random_state=7, reg_alpha=0.464,
       reg_lambda=0.8571, scale_pos_weight=1, seed=None, silent=1,
       subsample=0.5213)

In [39]:
model_xgb.score(x_val, y_val)

0.8966792752039834

In [40]:
model_xgb.score(x_test, y_test)

0.8474775151201435

In [41]:
from sklearn.metrics import mean_squared_error as mse
from scipy.stats import pearsonr as r
print(mse(y_val, model_xgb.predict(x_val)))
print(mse(y_test, model_xgb.predict(x_test)))
r(y_test, model_xgb.predict(x_test).reshape(-1,1))[0]

0.0038498521348579607
0.006839117052433511


array([0.92082088])

In [42]:
y_pred_xgb = scaler_y.inverse_transform(model_xgb.predict(x_test).reshape(-1,1))
trace1 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_xgb.reshape(-1,),
                   name = "Extreme Gradient Boosting",
                   mode = 'markers',
                   line=dict(width=10),
                   marker = dict(
                    color = 'rgb(0, 150, 0)'))
data = [trace1]
layout = go.Layout(showlegend = True,
    xaxis=dict(
        title='Actual water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='Predicted water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [43]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [44]:
model_lgb.fit(x_train, y_train)
print(model_lgb.score(x_val, y_val))
print(model_lgb.score(x_test, y_test))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



0.8909851725957454
0.81666416220081


In [45]:
print(mse(y_val, model_lgb.predict(x_val)))
print(mse(y_test, model_lgb.predict(x_test)))
r(y_test, model_lgb.predict(x_test).reshape(-1,1))[0]

0.004062021117660822
0.008220789581302058


array([0.90375041])

In [46]:
y_pred_lgb = scaler_y.inverse_transform(model_lgb.predict(x_test).reshape(-1,1))
trace1 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_lgb.reshape(-1,),
                   name = "Actual",
                   mode = 'markers',
                   line=dict(width=10),
                   marker = dict(color = 'rgb(0,150,0)'))
data = [trace1]
layout = go.Layout(
    xaxis=dict(
        title='Actual water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='Predicted water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [47]:
model_ett = ExtraTreesRegressor()
model_ett.fit(x_train, y_train)
model_ett.score(x_val, y_val)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



0.9240015180946729

In [48]:
model_ett.score(x_test, y_test)

0.861446467738461

In [49]:
print(mse(y_val, model_ett.predict(x_val)))
print(mse(y_test, model_ett.predict(x_test)))
r(y_test, model_ett.predict(x_test).reshape(-1,1))[0]

0.002831793121726801
0.006212748408283598


array([0.92836389])

In [50]:
y_pred_ett = scaler_y.inverse_transform(model_ett.predict(x_test).reshape(-1,1))
trace1 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_ett.reshape(-1,),
                   name = "Extra Trees Regression",
                   mode = 'markers',
                   line=dict(width=10),
                   marker = dict(color = 'rgb(0,150,0)'))
data = [trace1]
layout = go.Layout(showlegend = True,
    xaxis=dict(
        title='Actual water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='Predicted gas rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [51]:
model_knr = KNeighborsRegressor(n_neighbors = 3)
model_knr.fit(x_train, y_train)
model_knr.score(x_val, y_val)

0.8958781276614258

In [52]:
model_knr.score(x_test, y_test)

0.8475777994480822

In [53]:
print(mse(y_val, model_knr.predict(x_val)))
print(mse(y_test, model_knr.predict(x_test)))
r(y_test, model_knr.predict(x_test).reshape(-1,1))[0]

0.003879703837729192
0.0068346202973626905


array([0.92089061])

In [54]:
y_pred_knr = scaler_y.inverse_transform(model_knr.predict(x_test).reshape(-1,1))
trace1 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_knr.reshape(-1,),
                   name = "Actual",
                   mode = 'markers',
                   line=dict(width=10),
                   marker = dict(color = 'rgb(255,0,0)'))
data = [trace1]
layout = go.Layout(
    xaxis=dict(
        title='Actual water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='Predicted water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [55]:
model_dct = DecisionTreeRegressor()
model_dct.fit(x_train, y_train)
model_dct.score(x_val, y_val)

0.8874794189229281

In [56]:
model_dct.score(x_test, y_test)

0.8061760161047461

In [57]:
print(mse(y_val, model_dct.predict(x_val)))
print(mse(y_test, model_dct.predict(x_test)))
r(y_test, model_dct.predict(x_test).reshape(-1,1))[0]

0.004192649636655699
0.008691078659469819


array([0.90140859])

In [58]:
y_pred_dct = scaler_y.inverse_transform(model_dct.predict(x_test).reshape(-1,1))
trace1 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_dct.reshape(-1,),
                   name = "Decision Tree Regression",
                   mode = 'markers',
                   line=dict(width=10),
                   marker = dict(color = 'rgb(0,150,0)'))
data = [trace1]
layout = go.Layout(showlegend = True,
    xaxis=dict(
        title='Actual water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='Predicted water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [59]:
model_ridge = Ridge()
model_ridge.fit(x_train, y_train)
model_ridge.score(x_val, y_val)

0.4959650051388854

In [60]:
model_ridge.score(x_test, y_test)

0.5550057030941111

In [61]:
print(mse(y_val, model_ridge.predict(x_val)))
print(mse(y_test, model_ridge.predict(x_test)))
r(y_test, model_ridge.predict(x_test).reshape(-1,1))[0]

0.018780938721056992
0.019953570036588486


array([0.74923885])

In [62]:
y_pred_ridge = scaler_y.inverse_transform(model_ridge.predict(x_test).reshape(-1,1))
trace1 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_ridge.reshape(-1,),
                   name = "Actual",
                   mode = 'markers',
                   line=dict(width=10),
                   marker = dict(color = 'rgb(255,0,0)'))
data = [trace1]
layout = go.Layout(
    xaxis=dict(
        title='Actual water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='Predicted water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)

In [63]:
model_svr = SVR()
model_svr.fit(x_train, y_train)
model_svr.score(x_val, y_val)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



0.7066874729797217

In [64]:
model_svr.score(x_test, y_test)

0.7060629525928825

In [65]:
print(mse(y_val, model_svr.predict(x_val)))
print(mse(y_test, model_svr.predict(x_test)))
r(y_test, model_svr.predict(x_test).reshape(-1,1))[0]

0.010929170895374287
0.013180154223473893


array([0.84197281])

In [66]:
from sklearn import svm, grid_search

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.



In [67]:
model_svr1 = SVR(kernel='rbf', C=1e1, gamma=10)
model_svr1.fit(x_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



SVR(C=10.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=10,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [68]:
model_svr1.score(x_val, y_val)

0.837205345691553

In [69]:
model_svr1.score(x_test, y_test)

0.8470501029925849

In [70]:
print(mse(y_val, model_svr1.predict(x_val)))
print(mse(y_test, model_svr1.predict(x_test)))
r(y_test, model_svr.predict(x_test).reshape(-1,1))[0]

0.006065920933773787
0.006858282236978632


array([0.84197281])

In [71]:
trace2 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_gbm.reshape(-1,),
                   name = "Gradient Boosting Machine",
                   mode = 'markers',
                   line=dict(width=10))
trace3 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_rf.reshape(-1,),
                   name = "Random Forest",
                   mode = 'markers',
                   line=dict(width=10))
trace4 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_xgb.reshape(-1,),
                   name = "Extreme Gradient Boosting",
                   mode = 'markers',
                   line=dict(width=10))
trace5 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_ett.reshape(-1,),
                   name = "Extra Trees Regression",
                   mode = 'markers',
                   line=dict(width=10))
trace6 = go.Scatter(x = y_act.reshape(-1,),
                    y = y_pred_dct.reshape(-1,),
                   name = "Decision Tree Regression",
                   mode = 'markers',
                   line=dict(width=10))
data = [ trace2, trace3, trace4, trace5, trace6]
layout = go.Layout(showlegend = True,
    xaxis=dict(
        title='Actual water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='Predicted water rate (m3)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            #color='lightgrey'
        ),
        showticklabels=True,
        tickangle=45,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=data, layout = layout)
py.iplot(fig)